In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup, element
import urllib
import time

Rank - Ranking of overall sales

Name - The games name

Platform - Platform of the games release (i.e. PC,PS4, etc.)

Year - Year of the game's release

Genre - Genre of the game

Publisher - Publisher of the game

NA_Sales - Sales in North America (in millions)

EU_Sales - Sales in Europe (in millions)

JP_Sales - Sales in Japan (in millions)

Other_Sales - Sales in the rest of the world (in millions)

Global_Sales - Total worldwide sales.

In [97]:
df = pd.read_csv("vgsales.csv")
df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [5]:
df.describe()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,16598.000000,16327.000000,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000
mean,8300.605254,2006.406443,0.264667,0.146652,0.077782,0.048063,0.537441
std,4791.853933,5.828981,0.816683,0.505351,0.309291,0.188588,1.555028
min,1.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,4151.250000,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000
50%,8300.500000,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000
75%,12449.750000,2010.000000,0.240000,0.110000,0.040000,0.040000,0.470000
max,16600.000000,2020.000000,41.490000,29.020000,10.220000,10.570000,82.740000


In [6]:
df.isnull().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

# **clean Year**

In [7]:
df[df['Year'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
179,180,Madden NFL 2004,PS2,NaN,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23
377,378,FIFA Soccer 2004,PS2,NaN,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49
431,432,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,1.86,1.02,0.00,0.29,3.17
470,471,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,NaN,1.57,1.02,0.00,0.41,3.00
607,608,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0.00,0.03,2.53
...,...,...,...,...,...,...,...,...,...,...,...
16307,16310,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0.00,0.00,0.00,0.01
16327,16330,Inversion,PC,NaN,Shooter,Namco Bandai Games,0.01,0.00,0.00,0.00,0.01
16366,16369,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,Unknown,0.01,0.00,0.00,0.00,0.01
16427,16430,Virtua Quest,GC,NaN,Role-Playing,Unknown,0.01,0.00,0.00,0.00,0.01


In [9]:
#scrape data from website for "year"
def scrape_year(game_name, target_plt):
  print("games = ", game_name, "Platform", target_plt)
  #scrape game data
  rec_count = 0
  gname = []
  platform = []
  year = []

  urlhead = 'http://www.vgchartz.com/gamedb/games.php?name='
  urltail = game_name
  urltail += '&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showcriticscore=1&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showuserscore=1&showjapansales=0&showjapansales=1&showlastupdate=0&showothersales=0&showothersales=1&showshipped=0'


  surl = urlhead + urltail
  r = urllib.request.urlopen(surl).read()
  soup = BeautifulSoup(r)


  # vgchartz website is really weird so we have to search for
  # <a> tags with game urls
  game_tags = list(filter(
    #lambda x: x.attrs['href'].startswith('http://www.vgchartz.com/game/'),
    lambda x: x.get('href').startswith('https://www.vgchartz.com/game/'),
    # discard the first 10 elements because those
    # links are in the navigation bar
    soup.find_all("a", href=True)
  ))


  #print(surl)
  #print(game_name)
  #print(target_plt)
  #print(game_tags)

  if not game_tags:
    print(game_name, "is wrong")
    return

  for tag in game_tags:
    # get different attributes
    # traverse up the DOM tree
    data = tag.parent.parent.find_all("td")

    if data[3].find('img').attrs['alt'] == target_plt:
      # add name to list
      gname.append(" ".join(tag.string.split()))
      #print(f"{rec_count + 1} Fetch data for game {gname[-1]}")

      platform.append(data[3].find('img').attrs['alt'])
      release_year = data[13].string.split()[-1]

      # different format for year
      if release_year.startswith('N/A'):
          year.append(np.nan)
      else:
          if int(release_year) >= 50:
              year_to_add = np.int32("19" + release_year)
          else:
              year_to_add = np.int32("20" + release_year)
          year.append(year_to_add)


    rec_count += 1
    
    time.sleep(.3)

  columns = {
      'Name': gname,
      'Platform': platform,
      'Year': year,
  }
  
  if len(columns['Name']) == 0:
    return np.nan

  df_nan = pd.DataFrame(columns)
  df_nan = df_nan[['Name', 'Platform', 'Year']]
  #print(df_nan)
  print(df_nan[df_nan['Platform'] == target_plt]['Year'].values)
  return df_nan[df_nan['Platform'] == target_plt]['Year'].values[0]


In [ ]:
year_nan_df = df[df['Year'].isnull()][['Name', 'Platform']]
year_nan_name = year_nan_df['Name'].values
year_nan_plt = year_nan_df['Platform'].values

#transform game name to url format
func = np.vectorize(urllib.parse.quote)
year_nan_name = np.array(func(year_nan_name, safe=''))

'''func_scrape = np.vectorize(scrape_year)
missing_year = func_scrape(year_nan_name, year_nan_plt)'''

#get missing year 
missing_year = [] 
for i in range(len(year_nan_name)):
  missing_year.append(scrape_year(year_nan_name[i], year_nan_plt[i]))



In [98]:
#replace nan data
missing_year = np.array(missing_year)
print(missing_year.shape)

index_year_nan = df[df['Year'].isnull()]["Year"].index

for i in range(len(index_year_nan)):
  df.loc[index_year_nan[i], ('Year')] = missing_year[i]
  


(271,)


In [99]:
print(np.unique(df['Year']))
print("still have missing data", df['Year'].isnull().sum())

[1970. 1977. 1978. 1979. 1980. 1981. 1982. 1983. 1984. 1985. 1986. 1987.
 1988. 1989. 1990. 1991. 1992. 1993. 1994. 1995. 1996. 1997. 1998. 1999.
 2000. 2001. 2002. 2003. 2004. 2005. 2006. 2007. 2008. 2009. 2010. 2011.
 2012. 2013. 2014. 2015. 2016. 2017. 2020.   nan   nan   nan   nan   nan
   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
   nan   nan   nan   nan   nan   nan]
still have missing data 47


# **clean Publisher column**

In [89]:
#scrape website for publisher information
def scrape_publisher(game_name, target_plt):
  print("games = ", game_name, "Platform", target_plt)
  #scrape game data
  gname = []
  platform = []
  publisher = []

  urlhead = 'http://www.vgchartz.com/gamedb/games.php?name='
  urltail = game_name
  urltail += '&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showcriticscore=1&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showuserscore=1&showjapansales=0&showjapansales=1&showlastupdate=0&showothersales=0&showothersales=1&showshipped=0'


  surl = urlhead + urltail
  r = urllib.request.urlopen(surl).read()
  soup = BeautifulSoup(r)


  # vgchartz website is really weird so we have to search for
  # <a> tags with game urls
  game_tags = list(filter(
    #lambda x: x.attrs['href'].startswith('http://www.vgchartz.com/game/'),
    lambda x: x.get('href').startswith('https://www.vgchartz.com/game/'),
    # discard the first 10 elements because those
    # links are in the navigation bar
    soup.find_all("a", href=True)
  ))


  #print(surl)
  #print(game_name)
  #print(target_plt)
  #print(game_tags)

  if not game_tags:
    print(game_name, "is wrong")
    return

  for tag in game_tags:
    # get different attributes
    # traverse up the DOM tree
    data = tag.parent.parent.find_all("td")

    if data[3].find('img').attrs['alt'] == target_plt:
      # add name to list
      gname.append(" ".join(tag.string.split()))
      #print(f"{rec_count + 1} Fetch data for game {gname[-1]}")

      platform.append(data[3].find('img').attrs['alt'])
      publisher.append(data[4].string)
    
    time.sleep(.3)

  columns = {
      'Name': gname,
      'Platform': platform,
      'Publisher': publisher,
  }
  
  if len(columns['Name']) == 0:
    return np.nan

  df_nan = pd.DataFrame(columns)
  df_nan = df_nan[['Name', 'Platform', 'Publisher']]
  #print(df_nan)
  print(df_nan[df_nan['Platform'] == target_plt]['Publisher'].values)
  return df_nan[df_nan['Platform'] == target_plt]['Publisher'].values[0]


In [ ]:
pub_nan_df = df[df['Publisher'].isnull()][['Name', 'Platform']]
pub_nan_name = pub_nan_df['Name'].values
pub_nan_plt = pub_nan_df['Platform'].values

#transform game name to url format
pub_nan_name = np.array(func(pub_nan_name, safe=''))

missing_pub = [] 
for i in range(len(pub_nan_name)):
  missing_pub.append(scrape_year(pub_nan_name[i], pub_nan_plt[i]))
print(missing_pub)


In [100]:
#replace nan data
missing_pub = np.array(missing_pub)
print(missing_pub.shape)

index_pub_nan = df[df['Publisher'].isnull()]["Publisher"].index

for i in range(len(index_pub_nan)):
  df.loc[index_pub_nan[i], ('Publisher')] = missing_pub[i]
  


(58,)


In [101]:
print("still have missing value: ", df['Publisher'].isnull().sum())

12

In [108]:
df[df['Publisher'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
2222,2224,Bentley's Hackpack,GBA,2005.0,Misc,NaN,0.67,0.25,0.00,0.02,0.93
5302,5304,Dragon Ball Z: Budokai Tenkaichi 2 (JP sales),Wii,NaN,Action,NaN,0.15,0.05,0.14,0.01,0.35
6272,6274,The Legend of Zelda: The Minish Cap(weekly JP ...,GBA,NaN,Action,None,0.00,0.00,0.27,0.01,0.27
7351,7353,Yu Yu Hakusho: Dark Tournament,PS2,NaN,Fighting,NaN,0.10,0.08,0.00,0.03,0.21
7953,7955,Thomas the Tank Engine & Friends,GBA,2004.0,Adventure,NaN,0.13,0.05,0.00,0.00,0.19
10494,10496,Atsumare! Power Pro Kun no DS Koushien,DS,2006.0,Sports,NaN,0.00,0.00,0.10,0.00,0.10
11076,11078,Action Man-Operation Extreme,PS,NaN,Action,None,0.05,0.03,0.00,0.01,0.09
12487,12489,Chou Soujuu Mecha MG,DS,NaN,Simulation,NaN,0.00,0.00,0.06,0.00,0.06
13278,13280,Monster Hunter Frontier Online,PS3,NaN,Role-Playing,NaN,0.00,0.00,0.05,0.00,0.05
13672,13674,B.L.U.E.: Legend of Water,PS,NaN,Adventure,NaN,0.00,0.00,0.04,0.00,0.04


In [109]:
df[df['Publisher'] == "Unknown"]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
940,942,Gran Turismo Concept 2001 Tokyo,PS2,2001.0,Racing,Unknown,0.00,1.10,0.42,0.33,1.84
1649,1651,NASCAR Thunder 2003,PS2,2002.0,Racing,Unknown,0.60,0.46,0.00,0.16,1.22
2114,2116,Suikoden III,PS2,2002.0,Role-Playing,Unknown,0.29,0.23,0.38,0.08,0.98
2173,2175,Wheel of Fortune,PS2,2003.0,Misc,Unknown,0.47,0.36,0.00,0.12,0.95
2236,2238,Teenage Mutant Ninja Turtles,GBA,2003.0,Action,Unknown,0.67,0.25,0.00,0.02,0.93
...,...,...,...,...,...,...,...,...,...,...,...
16460,16463,"Horse Life 4: My Horse, My Friend, My Champion",3DS,2015.0,Action,Unknown,0.00,0.01,0.00,0.00,0.01
16493,16496,The Smurfs,3DS,NaN,Action,Unknown,0.00,0.01,0.00,0.00,0.01
16532,16535,STORM: Frontline Nation,PC,2011.0,Strategy,Unknown,0.00,0.01,0.00,0.00,0.01
16545,16548,The Treasures of Mystery Island 3 Pack - Save ...,PC,2011.0,Puzzle,Unknown,0.01,0.00,0.00,0.00,0.01


# **summary**

In [102]:
df.isnull().sum()

Rank             0
Name             0
Platform         0
Year            47
Genre            0
Publisher       12
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
Global_Sales     0
dtype: int64

In [103]:
df[df['Year'].isnull()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
5302,5304,Dragon Ball Z: Budokai Tenkaichi 2 (JP sales),Wii,NaN,Action,NaN,0.15,0.05,0.14,0.01,0.35
6272,6274,The Legend of Zelda: The Minish Cap(weekly JP ...,GBA,NaN,Action,None,0.00,0.00,0.27,0.01,0.27
6496,6498,Jonah Lomu Rugby Challenge,PS3,NaN,Sports,Home Entertainment Suppliers,0.00,0.20,0.00,0.06,0.26
7351,7353,Yu Yu Hakusho: Dark Tournament,PS2,NaN,Fighting,NaN,0.10,0.08,0.00,0.03,0.21
7423,7425,WRC: FIA World Rally Championship,PS3,NaN,Racing,Black Bean Games,0.00,0.15,0.01,0.05,0.21
7615,7617,Famista 64,N64,NaN,Sports,Namco Bandai Games,0.00,0.00,0.17,0.03,0.20
8376,8378,Sword of the Samurai,PS2,NaN,Fighting,Ubisoft,0.00,0.00,0.17,0.00,0.17
9522,9524,Nintendo Puzzle Collection,GC,NaN,Puzzle,Unknown,0.00,0.00,0.13,0.00,0.13
9692,9694,WRC: FIA World Rally Championship,X360,NaN,Racing,Black Bean Games,0.00,0.10,0.00,0.02,0.12
10272,10274,GiFTPiA,GC,NaN,Role-Playing,Unknown,0.00,0.00,0.11,0.00,0.11


In [110]:
df.dropna(inplace=True)

In [111]:
df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [114]:
df.isnull().sum()

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64

In [116]:
df.to_csv("vgsales_clean.csv", sep=",", encoding='utf-8', index=False)

In [115]:
print(df.shape[0])
df['Publisher'] = df['Publisher'].astype('str')

for i in df.columns:
  print("{0:14s} unique value = {1}".format(i, len(np.unique(df[i]))))

16548
Rank           unique value = 16548
Name           unique value = 11458
Platform       unique value = 31
Year           unique value = 43
Genre          unique value = 12
Publisher      unique value = 590
NA_Sales       unique value = 409
EU_Sales       unique value = 305
JP_Sales       unique value = 244
Other_Sales    unique value = 157
Global_Sales   unique value = 623
